In [1]:
import keras

Using TensorFlow backend.


In [2]:
from keras import models
from keras import layers
from keras import applications
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
import numpy as np
import cPickle as pickle
K.set_image_dim_ordering('th')
# load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# reshape to be [samples][pixels][width][height]
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')
np.random.seed(6)
# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255
y_train = y_train / 10.
y_test = y_test / 10.
num_classes = 1
X_train = np.squeeze(X_train, axis=1)
X_test = np.squeeze(X_test, axis=1)
X_train = np.pad(X_train, ((0, 0), (10, 10), (10, 10)), 'constant')
X_test = np.pad(X_test, ((0, 0), (10, 10), (10, 10)), 'constant')

In [3]:
X_train_3D = np.stack((X_train,)*3, -1)
X_test_3D = np.stack((X_test,)*3, -1)
print(X_train_3D.shape)
X_train_3D = np.transpose(X_train_3D, (0, 3, 1, 2))
X_test_3D = np.transpose(X_test_3D, (0, 3, 1, 2))
print(X_train_3D.shape)

feature_generator = applications.VGG19(include_top=False, weights=None, input_shape=(3,48,48))
X_train_3D_features = feature_generator.predict(X_train_3D, verbose=True)
X_test_3D_features = feature_generator.predict(X_test_3D, verbose=True)
feature_shape = X_train_3D_features.shape[1] * X_train_3D_features.shape[2] * X_train_3D_features.shape[3]
X_train = X_train_3D_features.reshape(len(X_train_3D_features), feature_shape)
X_test = X_test_3D_features.reshape(len(X_test_3D_features), feature_shape)

(60000, 48, 48, 3)
(60000, 3, 48, 48)
10000/10000 [==============================] - 10s 970us/step


In [4]:
MLP = models.Sequential()
MLP.add(layers.Flatten(input_shape=feature_generator.output_shape[1:]))
MLP.add(layers.Dense(256, activation='relu', input_dim=(3,48,48)))
MLP.add(layers.Dropout(0.2))
MLP.add(layers.Dense(1, activation='linear')) # REGRESSION <--- HERE WAS 128 but for regression it should be 1!!!

In [5]:
model = keras.Model(inputs=feature_generator.input, outputs=MLP(feature_generator.output))

In [6]:
from keras import optimizers
sgd = optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_squared_error', optimizer=sgd, metrics=['mse', 'mae']) 

In [7]:
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto'), \
            keras.callbacks.ModelCheckpoint('/tmp/VGG_MNIST_modelfile.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')]

In [8]:
model.fit(X_train_3D, y_train, validation_data=(X_test_3D, y_test), callbacks=callbacks, epochs=1000, batch_size=32, verbose=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1000
60000/60000 [==============================] - 209s 3ms/step - loss: 0.0932 - mean_squared_error: 0.0932 - mean_absolute_error: 0.2617 - val_loss: 0.0837 - val_mean_squared_error: 0.0837 - val_mean_absolute_error: 0.2533

Epoch 00001: val_loss improved from inf to 0.08366, saving model to /tmp/VGG_MNIST_modelfile.h5
Epoch 2/1000
60000/60000 [==============================] - 212s 4ms/step - loss: 0.0836 - mean_squared_error: 0.0836 - mean_absolute_error: 0.2525 - val_loss: 0.0836 - val_mean_squared_error: 0.0836 - val_mean_absolute_error: 0.2533

Epoch 00002: val_loss improved from 0.08366 to 0.08357, saving model to /tmp/VGG_MNIST_modelfile.h5
Epoch 3/1000
60000/60000 [==============================] - 215s 4ms/step - loss: 0.0836 - mean_squared_error: 0.0836 - mean_absolute_error: 0.2524 - val_loss: 0.0835 - val_mean_squared_error: 0.0835 - val_mean_absolute_error: 0.2530

Epoch 00003: val_loss improved from 0.08357 to 0.

60000/60000 [==============================] - 212s 4ms/step - loss: 0.0138 - mean_squared_error: 0.0138 - mean_absolute_error: 0.0830 - val_loss: 0.0117 - val_mean_squared_error: 0.0117 - val_mean_absolute_error: 0.0743

Epoch 00025: val_loss improved from 0.01290 to 0.01170, saving model to /tmp/VGG_MNIST_modelfile.h5
Epoch 26/1000
60000/60000 [==============================] - 212s 4ms/step - loss: 0.0127 - mean_squared_error: 0.0127 - mean_absolute_error: 0.0790 - val_loss: 0.0110 - val_mean_squared_error: 0.0110 - val_mean_absolute_error: 0.0714

Epoch 00026: val_loss improved from 0.01170 to 0.01098, saving model to /tmp/VGG_MNIST_modelfile.h5
Epoch 27/1000
60000/60000 [==============================] - 210s 4ms/step - loss: 0.0120 - mean_squared_error: 0.0120 - mean_absolute_error: 0.0760 - val_loss: 0.0103 - val_mean_squared_error: 0.0103 - val_mean_absolute_error: 0.0678

Epoch 00027: val_loss improved from 0.01098 to 0.01031, saving model to /tmp/VGG_MNIST_modelfile.h5
Epoch 


Epoch 00050: val_loss improved from 0.00576 to 0.00567, saving model to /tmp/VGG_MNIST_modelfile.h5
Epoch 51/1000
60000/60000 [==============================] - 208s 3ms/step - loss: 0.0062 - mean_squared_error: 0.0062 - mean_absolute_error: 0.0514 - val_loss: 0.0056 - val_mean_squared_error: 0.0056 - val_mean_absolute_error: 0.0442

Epoch 00051: val_loss improved from 0.00567 to 0.00555, saving model to /tmp/VGG_MNIST_modelfile.h5
Epoch 52/1000
60000/60000 [==============================] - 210s 4ms/step - loss: 0.0061 - mean_squared_error: 0.0061 - mean_absolute_error: 0.0510 - val_loss: 0.0058 - val_mean_squared_error: 0.0058 - val_mean_absolute_error: 0.0457

Epoch 00052: val_loss did not improve from 0.00555
Epoch 53/1000
60000/60000 [==============================] - 210s 4ms/step - loss: 0.0060 - mean_squared_error: 0.0060 - mean_absolute_error: 0.0506 - val_loss: 0.0063 - val_mean_squared_error: 0.0063 - val_mean_absolute_error: 0.0494

Epoch 00053: val_loss did not improve fr

60000/60000 [==============================] - 210s 4ms/step - loss: 0.0044 - mean_squared_error: 0.0044 - mean_absolute_error: 0.0435 - val_loss: 0.0044 - val_mean_squared_error: 0.0044 - val_mean_absolute_error: 0.0382

Epoch 00077: val_loss improved from 0.00446 to 0.00435, saving model to /tmp/VGG_MNIST_modelfile.h5
Epoch 78/1000
60000/60000 [==============================] - 210s 4ms/step - loss: 0.0043 - mean_squared_error: 0.0043 - mean_absolute_error: 0.0431 - val_loss: 0.0044 - val_mean_squared_error: 0.0044 - val_mean_absolute_error: 0.0381

Epoch 00078: val_loss did not improve from 0.00435
Epoch 79/1000
60000/60000 [==============================] - 210s 4ms/step - loss: 0.0042 - mean_squared_error: 0.0042 - mean_absolute_error: 0.0427 - val_loss: 0.0045 - val_mean_squared_error: 0.0045 - val_mean_absolute_error: 0.0405

Epoch 00079: val_loss did not improve from 0.00435
Epoch 80/1000
60000/60000 [==============================] - 210s 4ms/step - loss: 0.0042 - mean_squared


Epoch 00103: val_loss improved from 0.00384 to 0.00382, saving model to /tmp/VGG_MNIST_modelfile.h5
Epoch 104/1000
60000/60000 [==============================] - 210s 4ms/step - loss: 0.0034 - mean_squared_error: 0.0034 - mean_absolute_error: 0.0388 - val_loss: 0.0042 - val_mean_squared_error: 0.0042 - val_mean_absolute_error: 0.0376

Epoch 00104: val_loss did not improve from 0.00382
Epoch 105/1000
60000/60000 [==============================] - 210s 3ms/step - loss: 0.0033 - mean_squared_error: 0.0033 - mean_absolute_error: 0.0385 - val_loss: 0.0038 - val_mean_squared_error: 0.0038 - val_mean_absolute_error: 0.0350

Epoch 00105: val_loss improved from 0.00382 to 0.00380, saving model to /tmp/VGG_MNIST_modelfile.h5
Epoch 106/1000
60000/60000 [==============================] - 208s 3ms/step - loss: 0.0033 - mean_squared_error: 0.0033 - mean_absolute_error: 0.0382 - val_loss: 0.0040 - val_mean_squared_error: 0.0040 - val_mean_absolute_error: 0.0358

Epoch 00106: val_loss did not improve

60000/60000 [==============================] - 210s 4ms/step - loss: 0.0028 - mean_squared_error: 0.0028 - mean_absolute_error: 0.0355 - val_loss: 0.0036 - val_mean_squared_error: 0.0036 - val_mean_absolute_error: 0.0333

Epoch 00130: val_loss did not improve from 0.00351
Epoch 131/1000
60000/60000 [==============================] - 210s 4ms/step - loss: 0.0028 - mean_squared_error: 0.0028 - mean_absolute_error: 0.0355 - val_loss: 0.0036 - val_mean_squared_error: 0.0036 - val_mean_absolute_error: 0.0336

Epoch 00131: val_loss did not improve from 0.00351
Epoch 132/1000
60000/60000 [==============================] - 210s 4ms/step - loss: 0.0027 - mean_squared_error: 0.0027 - mean_absolute_error: 0.0352 - val_loss: 0.0035 - val_mean_squared_error: 0.0035 - val_mean_absolute_error: 0.0335

Epoch 00132: val_loss did not improve from 0.00351
Epoch 133/1000
60000/60000 [==============================] - 210s 4ms/step - loss: 0.0027 - mean_squared_error: 0.0027 - mean_absolute_error: 0.0352 -

60000/60000 [==============================] - 210s 4ms/step - loss: 0.0023 - mean_squared_error: 0.0023 - mean_absolute_error: 0.0328 - val_loss: 0.0034 - val_mean_squared_error: 0.0034 - val_mean_absolute_error: 0.0325

Epoch 00157: val_loss did not improve from 0.00333
Epoch 158/1000
60000/60000 [==============================] - 210s 4ms/step - loss: 0.0023 - mean_squared_error: 0.0023 - mean_absolute_error: 0.0330 - val_loss: 0.0034 - val_mean_squared_error: 0.0034 - val_mean_absolute_error: 0.0319

Epoch 00158: val_loss did not improve from 0.00333
Epoch 159/1000
60000/60000 [==============================] - 210s 4ms/step - loss: 0.0023 - mean_squared_error: 0.0023 - mean_absolute_error: 0.0327 - val_loss: 0.0033 - val_mean_squared_error: 0.0033 - val_mean_absolute_error: 0.0317

Epoch 00159: val_loss improved from 0.00333 to 0.00329, saving model to /tmp/VGG_MNIST_modelfile.h5
Epoch 160/1000
60000/60000 [==============================] - 207s 3ms/step - loss: 0.0023 - mean_squa

60000/60000 [==============================] - 208s 3ms/step - loss: 0.0020 - mean_squared_error: 0.0020 - mean_absolute_error: 0.0309 - val_loss: 0.0032 - val_mean_squared_error: 0.0032 - val_mean_absolute_error: 0.0314

Epoch 00184: val_loss did not improve from 0.00316
Epoch 185/1000
60000/60000 [==============================] - 208s 3ms/step - loss: 0.0020 - mean_squared_error: 0.0020 - mean_absolute_error: 0.0308 - val_loss: 0.0032 - val_mean_squared_error: 0.0032 - val_mean_absolute_error: 0.0306

Epoch 00185: val_loss improved from 0.00316 to 0.00316, saving model to /tmp/VGG_MNIST_modelfile.h5
Epoch 186/1000
60000/60000 [==============================] - 208s 3ms/step - loss: 0.0020 - mean_squared_error: 0.0020 - mean_absolute_error: 0.0309 - val_loss: 0.0032 - val_mean_squared_error: 0.0032 - val_mean_absolute_error: 0.0305

Epoch 00186: val_loss improved from 0.00316 to 0.00315, saving model to /tmp/VGG_MNIST_modelfile.h5
Epoch 187/1000
60000/60000 [=========================


Epoch 00210: val_loss did not improve from 0.00303
Epoch 211/1000
60000/60000 [==============================] - 208s 3ms/step - loss: 0.0017 - mean_squared_error: 0.0017 - mean_absolute_error: 0.0293 - val_loss: 0.0031 - val_mean_squared_error: 0.0031 - val_mean_absolute_error: 0.0298

Epoch 00211: val_loss did not improve from 0.00303
Epoch 212/1000
60000/60000 [==============================] - 210s 4ms/step - loss: 0.0017 - mean_squared_error: 0.0017 - mean_absolute_error: 0.0291 - val_loss: 0.0032 - val_mean_squared_error: 0.0032 - val_mean_absolute_error: 0.0305

Epoch 00212: val_loss did not improve from 0.00303
Epoch 213/1000
60000/60000 [==============================] - 210s 4ms/step - loss: 0.0017 - mean_squared_error: 0.0017 - mean_absolute_error: 0.0290 - val_loss: 0.0031 - val_mean_squared_error: 0.0031 - val_mean_absolute_error: 0.0307

Epoch 00213: val_loss did not improve from 0.00303
Epoch 214/1000
60000/60000 [==============================] - 210s 4ms/step - loss: 

In [9]:
y_pred = model.predict(X_test_3D)

In [10]:
y_pred = np.ndarray.flatten(y_pred)

In [11]:
stats = (y_test, y_pred)
with open('Stats_Files/mnvggreg.p', 'w') as f:
    pickle.dump(stats, f)